In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Input, Embedding
model = Sequential()
model.add(Input(shape=(80,))) # 입력하는 영화평의 길이를 80으로 제한, 길면 자르고, 짧으면 zero padding
model.add(Embedding(input_dim=10000, output_dim=32))
model.add(SimpleRNN(64))
model.add(Dense(2, activation='softmax')) # model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 32)            320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                6208      
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 326338 (1.24 MB)
Trainable params: 326338 (1.24 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [2]:
# model.compile(loss='binary_crossentropy', optimizer='adam',
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [3]:
from tensorflow.keras.datasets import imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(25000,) (25000,) (25000,) (25000,)


In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_train, maxlen=80, truncating='post', padding='post')
X_test_pad = pad_sequences(X_test, maxlen=80, truncating='post', padding='post')

In [5]:
%%time
model.fit(X_train_pad, y_train, epochs=10, batch_size=200)

Epoch 1/10
125/125 [==============================] - 3s 19ms/step - loss: 0.5728 - accuracy: 0.6760
Epoch 2/10
125/125 [==============================] - 2s 19ms/step - loss: 0.3540 - accuracy: 0.8528
Epoch 3/10
125/125 [==============================] - 2s 19ms/step - loss: 0.2496 - accuracy: 0.9039
Epoch 4/10
125/125 [==============================] - 2s 18ms/step - loss: 0.1502 - accuracy: 0.9495
Epoch 5/10
125/125 [==============================] - 2s 19ms/step - loss: 0.0714 - accuracy: 0.9780
Epoch 6/10
125/125 [==============================] - 2s 19ms/step - loss: 0.0274 - accuracy: 0.9934
Epoch 7/10
125/125 [==============================] - 2s 19ms/step - loss: 0.0170 - accuracy: 0.9956
Epoch 8/10
125/125 [==============================] - 2s 20ms/step - loss: 0.0126 - accuracy: 0.9971
Epoch 9/10
125/125 [==============================] - 2s 19ms/step - loss: 0.0048 - accuracy: 0.9994
Epoch 10/10
125/125 [==============================] - 2s 19ms/step - loss: 0.0024 - accura

In [6]:
model.evaluate(X_test_pad, y_test)

782/782 [==============================] - 2s 2ms/step - loss: 1.2416 - accuracy: 0.7720


[1.2415971755981445, 0.7719600200653076]

In [7]:
print(X_test_pad.shape)

(25000, 80)


In [8]:
import numpy as np
pred = model.predict(X_test_pad)
# pred = (pred > 0.5).astype(int)
pred = np.argmax(pred, axis=1)

782/782 [==============================] - 2s 2ms/step


In [9]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, pred))

[[9820 2680]
 [3021 9479]]


In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.77196

In [ ]:
# 옵티마이저를 sgd로 바꿔보세요. accuracy: 0.57784
# 전체 단어의 개수를 1000개로 바꿔보세요. accuracy:
# 영화평의 길이를 200개로 바꿔보세요. accuracy:
# pad_sequence의 truncating과 padding을 pre로 바꿔보세요. accuracy:

In [12]:
# 아래 택스트를 긍/부정 분류하세요.
text = "My God the actors who potrayed the VIP people cannot act. I cringed everytime they said a line. It felt like they were just reading them. Even the intonation was off. It was like when we were kids and had to read a play in class and we exagerated the intonation. Terrible, just awful."

In [13]:
word_index = imdb.get_word_index()
# print(word_index)
# {'fawn': 34701, 'tsukino': 52006, 'nunnery': 52007,

In [14]:
word_to_idx = {k:(v+3) for k,v in word_index.items()}
word_to_idx["<PAD>"] = 0
word_to_idx["<START>"] = 1
word_to_idx["<UNK>"] = 2  # unknown
word_to_idx["<UNUSED>"] = 3

In [16]:
input_text = text.lower().split()
print(input_text)

['my', 'god', 'the', 'actors', 'who', 'potrayed', 'the', 'vip', 'people', 'cannot', 'act.', 'i', 'cringed', 'everytime', 'they', 'said', 'a', 'line.', 'it', 'felt', 'like', 'they', 'were', 'just', 'reading', 'them.', 'even', 'the', 'intonation', 'was', 'off.', 'it', 'was', 'like', 'when', 'we', 'were', 'kids', 'and', 'had', 'to', 'read', 'a', 'play', 'in', 'class', 'and', 'we', 'exagerated', 'the', 'intonation.', 'terrible,', 'just', 'awful.']


In [17]:
# 단어를 숫자로 변환합니다. 
# word_to_idx에 없는 단어는 2(<UNK>)로 지정하며, 
# 인덱스가 10000보다 크면 3(<UNUSED>)로 지정합니다.
def encoding(review_text):
  encoded = []
  for word in review_text:
    try:
      idx = word_to_idx[word]
      if idx>10000:
        encoded.append(3)
      else:
        encoded.append(idx)
    except:
      encoded.append(2)
  return encoded

input_encoded = encoding(input_text)

In [18]:
np.array(input_encoded)[np.newaxis, :]

array([[ 61, 558,   4, 156,  37,   2,   4,   3,  84, 566,   2,  13,   3,
          3,  36, 301,   6,   2,  12, 421,  40,  36,  71,  43, 886,   2,
         60,   4,   3,  16,   2,  12,  16,  40,  54,  75,  71, 362,   5,
         69,   8, 332,   6, 297,  11, 707,   5,  75,   3,   4,   2,   2,
         43,   2]])

In [20]:
input_pad = pad_sequences(
    np.array(input_encoded)[np.newaxis, :],
    maxlen=80, truncating='post', padding='post')

In [ ]:
model.predict(input_pad) # LSTM을 이용한 인공신경망 모형은 부정(0)으로 분류함